<a href="https://colab.research.google.com/github/maleeham/Practical-Aspects-of-Data-Science/blob/master/Dog_Breed_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project 1: CS 6301**
##*Practical Aspects of Data Science* 


---

###### Shayan Sadar, sss170008
###### Maleeha Koul, msk180001

# **Dog Breed Identification** : Multinomial Classification Problem  
https://www.kaggle.com/c/dog-breed-identification




### Import the libraries

In [0]:
# Download all the packages
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from datetime import timedelta
import math
import os
import scipy.misc
from scipy.stats import itemfreq
from random import sample
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import PIL.Image
from IPython.display import display
import pandas as pd
from zipfile import ZipFile
from io import BytesIO

### Retrieve the train / test datasets from the server

In [21]:
!wget https://personal.utdallas.edu/~msk180001/train.zip
!wget https://personal.utdallas.edu/~msk180001/test.zip

--2020-03-05 00:00:25--  https://personal.utdallas.edu/~msk180001/train.zip
Resolving personal.utdallas.edu (personal.utdallas.edu)... 129.110.182.249
Connecting to personal.utdallas.edu (personal.utdallas.edu)|129.110.182.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 361647098 (345M) [application/zip]
Saving to: ‘train.zip.1’

train.zip.1          77%[==============>     ] 266.30M  1.72MB/s    eta 46s    ^C
--2020-03-05 00:03:01--  https://personal.utdallas.edu/~msk180001/test.zip
Resolving personal.utdallas.edu (personal.utdallas.edu)... 129.110.182.249
Connecting to personal.utdallas.edu (personal.utdallas.edu)|129.110.182.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 363111741 (346M) [application/zip]
Saving to: ‘test.zip.1’

test.zip.1            1%[                    ]   4.61M  1.51MB/s    eta 3m 46s ^C


#### Unzipping the data

In [22]:
!unzip train.zip
!unzip test.zip

Streaming output truncated to the last 5000 lines.
  inflating: test/09cbd98c55f58ad0535b5066c72e6e42.jpg  
  inflating: test/6f179916cdb7eeec8249b76e620171e2.jpg  
  inflating: test/5aa4280436034ebb01208a320ecda325.jpg  
  inflating: test/bbb501facf5f18996702587b1d69e186.jpg  
  inflating: test/0f3f071f6d0afc5bc906aee0cfb3297b.jpg  
  inflating: test/853eb43560635069afa5bce9b99da9dc.jpg  
  inflating: test/d359f629f70b113299e0235c2f8f8bda.jpg  
  inflating: test/d258058c7626e2ad6ed6d318de3b829e.jpg  
  inflating: test/269982792ab68ab8119de3219a3a286d.jpg  
  inflating: test/186ac5885342dfa37a76bfa1083798e6.jpg  
  inflating: test/b2eeb405c9902584f9b626a10fa52960.jpg  
  inflating: test/5fd4c3dad6367ee986e2f9c4a5358483.jpg  
  inflating: test/86a6ed1fcb3aa61db174097f89232715.jpg  
  inflating: test/12a5a3abbcb676f7eaf490073d30f978.jpg  
  inflating: test/950de996fe161ae9ceb4d0afd7c5011f.jpg  
  inflating: test/34a6fa4ebc327fc03164793636b1bfbe.jpg  
  inflating: test/e312dda8d35118c4088

In [23]:
archive_train = ZipFile("./train.zip", 'r')
archive_test = ZipFile("./test.zip", 'r')

len(archive_train.namelist()[:])-1 #we must remove the 1st value

10222

In [0]:
# This function help to create  a pickle file gathering all the image from a zip folder
def DataBase_creator(archivezip, nwidth, nheight, save_name):
    s = (len(archivezip.namelist()[:])-1, nwidth, nheight,3)
    allImage = np.zeros(s)

    for i in range(1,len(archivezip.namelist()[:])):
        filename = BytesIO(archivezip.read(archivezip.namelist()[i]))
        image = PIL.Image.open(filename)
        image = image.resize((nwidth, nheight))
        image = np.array(image)
        image = np.clip(image/255.0, 0.0, 1.0)

        allImage[i-1]=image
    
    pickle.dump(allImage, open( save_name + '.p', "wb" ) )

In [0]:
image_resize = 60

In [0]:
DataBase_creator(archivezip = archive_train, nwidth = image_resize, nheight = image_resize , save_name = "train")

In [0]:
DataBase_creator(archivezip = archive_test, nwidth = image_resize, nheight = image_resize , save_name = "test")

In [150]:
#load train data
train = pickle.load( open( "train.p", "rb" ) )
train.shape

(10222, 60, 60, 3)

In [151]:
#load test data 
test = pickle.load( open( "test.p", "rb" ) )
test.shape

(10357, 60, 60, 3)

#### Read the labels corresponding to the images in the train dataset

In [29]:
df_train= pd.read_csv('https://personal.utdallas.edu/~msk180001/labels.csv')
df_train.sample(5)

,id,breed
2771,443712eca3f249989cd7bf5c067dce12,dhole
846,14acf981af889d9b97d1d46a439228b0,tibetan_terrier
5618,8dc980c9f136b41bf36d0490cdad2ce8,maltese_dog
7080,b1d2238899cfa91f4da91994b304e090,brittany_spaniel
7591,be3ede4734509f6c5d5de4f40b867f9c,afghan_hound


In [0]:
nwidth = image_resize
nheight = image_resize
s = (len(df_train['breed']), nwidth, nheight,3)
allImage = np.zeros(s)
i= 0
for f, breed in df_train.values:
    image = PIL.Image.open('./train/{}.jpg'.format(f))
    image = image.resize((nwidth, nheight))
    image = np.array(image)
    image = np.clip(image/255.0, 0.0, 1.0)
    i += 1
    
    allImage[i-1]=image

train = allImage

In [153]:
labels_raw = pd.read_csv("https://personal.utdallas.edu/~msk180001/labels.csv", header=0, sep=',', quotechar='"')

labels_raw.sample(5)

,id,breed
7291,b6ac6d8b9923fc4441457305959ef92d,tibetan_terrier
863,15137b6e02d5cd04cabb34aba1fabb9f,chesapeake_bay_retriever
8343,d132a0af92ff56fe35d1811b8eae95f0,leonberg
7114,b27ccad9788e1868e258a96f8538830a,tibetan_mastiff
5771,91af3f0d18e5bd9b09a145ff5db9cb65,flat-coated_retriever


In [154]:
labels_raw = pd.read_csv('https://personal.utdallas.edu/~msk180001/labels.csv')
df_train.sample(5)

,id,breed
8381,d247349a06da617a7d7c54281121cebe,dandie_dinmont
6945,ae321243a4dfba310868a20dfaba514b,scotch_terrier
1344,2125684ed729de7ba10cc02ed8f75179,american_staffordshire_terrier
6777,aa4b973e445141931ec097aafa317395,clumber
6173,9b68b5724c63b5127703bce31dd1fa65,briard


#### Classifying 5 breeds from the entire dataset

In [156]:
Nber_of_breeds = 5

def main_breeds(labels_raw, Nber_breeds , all_breeds='TRUE'):
    labels_freq_pd = itemfreq(labels_raw["breed"])
    labels_freq_pd = labels_freq_pd[labels_freq_pd[:, 1].argsort()[::-1]] 
    
    if all_breeds == 'FALSE':
        main_labels = labels_freq_pd[:,0][0:Nber_breeds]
    else: 
        main_labels = labels_freq_pd[:,0][:]
        
    labels_raw_np = labels_raw["breed"].values 
    labels_raw_np = labels_raw_np.reshape(labels_raw_np.shape[0],1)

    labels_filtered_index = np.where(labels_raw_np == main_labels)
    
    return labels_filtered_index

labels_filtered_index = main_breeds(labels_raw = labels_raw, Nber_breeds = Nber_of_breeds, all_breeds='FALSE')
labels_filtered = labels_raw.iloc[labels_filtered_index[0],:]
train_filtered = train[labels_filtered_index[0],:,:,:]

print('Number of images remaining after selecting the {} main breeds : {}'.format(Nber_of_breeds, labels_filtered_index[0].shape))
print('The shape of train_filtered dataset is : {}'.format(train_filtered.shape))

Number of images remaining after selecting the 5 main breeds : (588,)
The shape of train_filtered dataset is : (588, 60, 60, 3)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  after removing the cwd from sys.path.


In [157]:
#Selecting the labels from the 'N' main breeds
labels = labels_filtered["breed"].values
labels = labels.reshape(labels.shape[0],1)
labels.shape

(588, 1)

#### One hot encoding the image labels

In [0]:
#Function to create one-hot labels
def matrix_Bin(labels):
    labels_bin=np.array([])

    labels_name, labels0 = np.unique(labels, return_inverse=True)
    labels0
    
    for _, i in enumerate(itemfreq(labels0)[:,0].astype(int)):
        labels_bin0 = np.where(labels0 == itemfreq(labels0)[:,0][i], 1., 0.)
        labels_bin0 = labels_bin0.reshape(1,labels_bin0.shape[0])

        if (labels_bin.shape[0] == 0):
            labels_bin = labels_bin0
        else:
            labels_bin = np.concatenate((labels_bin,labels_bin0 ),axis=0)

    print("Nber SubVariables {0}".format(itemfreq(labels0)[:,0].shape[0]))
    labels_bin = labels_bin.transpose()
    print("Shape : {0}".format(labels_bin.shape))
    
    return labels_name, labels_bin

In [159]:
labels_name, labels_bin = matrix_Bin(labels = labels)
labels_bin[0:9]

Nber SubVariables 5
Shape : (588, 5)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  app.launch_new_instance()


array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.]])

In [160]:
for breed in range(len(labels_name)):
    print('Breed {0} : {1}'.format(breed,labels_name[breed]))

Breed 0 : afghan_hound
Breed 1 : bernese_mountain_dog
Breed 2 : entlebucher
Breed 3 : maltese_dog
Breed 4 : scottish_deerhound


#### Split the dataset into train and test

In [161]:
num_validation = 0.30
X_train, X_validation, y_train, y_validation = train_test_split(train_filtered, labels_bin, test_size=num_validation, random_state=6)
X_train.shape

(411, 60, 60, 3)

In [162]:
def train_test_creation(x, data, toPred): 
    indices = sample(range(data.shape[0]),int(x * data.shape[0])) 
    indices = np.sort(indices, axis=None) 

    index = np.arange(data.shape[0]) 
    reverse_index = np.delete(index, indices,0)

    train_toUse = data[indices]
    train_toPred = toPred[indices]
    test_toUse = data[reverse_index]
    test_toPred = toPred[reverse_index]

    return train_toUse, train_toPred, test_toUse, test_toPred

df_train_toUse, df_train_toPred, df_test_toUse, df_test_toPred = train_test_creation(0.7, train_filtered, labels_bin)

df_validation_toPred_cls = np.argmax(y_validation, axis=1)
df_validation_toPred_cls[0:9]

array([1, 2, 3, 2, 3, 4, 3, 2, 0])

In [0]:
img_size = image_resize

num_channels = 3

img_size_flat = img_size * img_size

img_shape = (img_size, img_size, num_channels)

# Number of classes : 5 breeds
num_classes = Nber_of_breeds

#### Designing the Neural Network 

In [0]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

In [0]:
def new_conv_layer(input,              # The previous layer.
                   num_input_channels, # # channels in previous layer.
                   filter_size,        # Width and height of each filter.
                   num_filters,        # Number of filters.
                   use_pooling=True,
                   use_dropout=True):  # Use 2x2 max-pooling.

    shape = [filter_size, filter_size, num_input_channels, num_filters]

    weights = new_weights(shape=shape)

    biases = new_biases(length=num_filters)

    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')

    layer += biases

    if use_pooling:
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')

    layer = tf.nn.relu(layer)
    
    if use_dropout:
        layer = tf.nn.dropout(layer,keep_prob_conv)
    return layer, weights

In [0]:
def flatten_layer(layer):
    layer_shape = layer.get_shape()

    num_features = layer_shape[1:4].num_elements()
    
    layer_flat = tf.reshape(layer, [-1, num_features])

    return layer_flat, num_features

In [0]:
def new_fc_layer(input,          # The previous layer.
                 num_inputs,     # inputs from prev. layer.
                 num_outputs,    #  outputs.
                 use_relu=True,
                 use_dropout=True):

    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    layer = tf.matmul(input, weights) + biases

    if use_relu:
        layer = tf.nn.relu(layer)
    
    if use_dropout:
        layer = tf.nn.dropout(layer,keep_prob_fc)
        
    return layer

In [0]:
x = tf.placeholder(tf.float32, shape=[None, img_size, img_size, num_channels], name='x')
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels]) #-1 put everything as 1 array
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, axis=1)
keep_prob_fc=tf.placeholder(tf.float32)
keep_prob_conv=tf.placeholder(tf.float32)

#### Adding convolutional layers

In [44]:
# Convolutional Layer 1.
filter_size1 = 5          # Convolution filters are 5 x 5 pixels.
num_filters1 = 32         # There are 32 of these filters.


# Convolutional Layer 2.
filter_size2 = 4          # Convolution filters are 4 x 4 pixels.
num_filters2 = 64      # There are 64 of these filters.


# Convolutional Layer 3.
filter_size3 = 3          # Convolution filters are 3 x 3 pixels.
num_filters3 = 128      # There are 128 of these filters.


# Fully-connected layer.
fc_size = 500 

layer_conv1, weights_conv1 = \
    new_conv_layer(input=x_image,
                   num_input_channels=num_channels,
                   filter_size=filter_size1,
                   num_filters=num_filters1,
                   use_pooling=True,
                   use_dropout=False)
    
layer_conv2, weights_conv2 = \
    new_conv_layer(input=layer_conv1,
                   num_input_channels=num_filters1,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=True,
                   use_dropout=False)
    
layer_conv3, weights_conv3 = \
    new_conv_layer(input=layer_conv2,
                   num_input_channels=num_filters2,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=True,
                   use_dropout=True)

layer_flat, num_features = flatten_layer(layer_conv3)

#Train
layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=fc_size,
                         use_relu=True,
                         use_dropout=True)

layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=fc_size,
                         num_outputs=num_classes,
                         use_relu=False,
                         use_dropout=False)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


#### Prediction

In [0]:
y_pred = tf.nn.softmax(layer_fc2)
y_pred_cls = tf.argmax(y_pred, axis=1)

### Function Evaluation metrics 

In [0]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2, labels=y_true)

cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

correct_prediction = tf.equal(y_pred_cls, y_true_cls)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [0]:
session = tf.Session()
def init_variables():   
    session.run(tf.global_variables_initializer())

In [0]:
batch_size = 30

#function next_batch
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]


    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

#### Plotting the loss graph

In [0]:
def optimize(num_iterations, X):
    global total_iterations
    #array to plot
    losses = {'train':[], 'validation':[]}
    
    for i in range(num_iterations):
            total_iterations += 1
            x_batch, y_true_batch = next_batch(batch_size, X_train, y_train)

            feed_dict_train = {x: x_batch,
                               y_true: y_true_batch,
                               keep_prob_conv : 0.5,
                               keep_prob_fc : 0.6}
            feed_dict_validation = {x: X_validation,
                               y_true: y_validation,
                               keep_prob_conv : 1,
                               keep_prob_fc : 1}
            
            session.run(optimizer, feed_dict=feed_dict_train)
            
            acc_train = session.run(accuracy, feed_dict=feed_dict_train)
            acc_validation = session.run(accuracy, feed_dict=feed_dict_validation)
            losses['train'].append(acc_train)
            losses['validation'].append(acc_validation)
            
            # Print status every X iterations.
            if (total_iterations % X == 0) or (i ==(num_iterations -1)):
            # Calculate the accuracy on the training-set.
                                
                msg = "Iteration: {0:>6}, Training Accuracy: {1:>6.1%}, Validation Accuracy: {2:>6.1%}"
                print(msg.format(total_iterations, acc_train, acc_validation))
  
    
    plt.plot(losses['train'], label='Training accuracy')
    plt.plot(losses['validation'], label='Validation accuracy')
    
    plt.legend()
    _ = plt.ylim()

In [0]:
init_variables()

total_iterations = 0

optimize(num_iterations=1000, X=100)

Iteration:    100, Training Accuracy:  36.7%, Validation Accuracy:  36.7%
Iteration:    200, Training Accuracy:  63.3%, Validation Accuracy:  51.4%
Iteration:    300, Training Accuracy:  80.0%, Validation Accuracy:  61.6%
Iteration:    400, Training Accuracy:  70.0%, Validation Accuracy:  57.1%
Iteration:    500, Training Accuracy:  70.0%, Validation Accuracy:  59.9%


### Function for plotting the test images and showing the predicted and true labels

In [0]:
def plot_images(images, cls_true, cls_pred=None):
    assert len(images) == len(cls_true) == 25
    
    fig, axes = plt.subplots(5, 5)
    fig.subplots_adjust(left=2, bottom=2, right=3, top=3, wspace=1, hspace=0.3)


    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i].reshape(img_shape), cmap='binary')
        if cls_pred is None:
            xlabel = "True: {}".format(cls_true[i])
        else:
            xlabel = "True: {}, Pred: {}".format(cls_true[i] , cls_pred[i]) 
         

        ax.set_xlabel(xlabel)
        
        ax.set_xticks([])
        ax.set_yticks([])
    
    plt.show()

In [0]:
#Selecting random 25 images from the test set
feed_dict_validation = {x: X_validation, y_true: y_validation, keep_prob_conv : 1, keep_prob_fc : 1}

df_validation_Predicted_cls = session.run(y_pred_cls, feed_dict=feed_dict_validation)

plot_images(images=X_validation[37:62], 
            cls_true=df_validation_toPred_cls[37:62], 
            cls_pred=df_validation_Predicted_cls [37:62])